## SAT179_3_iNaturalist

* [#179](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/179) "Koppla SAT till artobservationer"
* notebooks
    * [SAT179_iNaturalist.ipynb](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/SAT179_iNaturalist.ipynb)
    * [SAT179_2_iNaturalist.ipynb](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/SAT179_2_iNaturalist.ipynb)
    * [SAT179_3_iNaturalist.ipynb](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/SAT179_3_iNaturalist.ipynb)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-23 15:54


In [3]:
"""
SAT iNaturalist – cleaned POC
---------------------------------
A tidy, single-file proof‑of‑concept that fetches recent iNaturalist observations
near the Stockholm Archipelago Trail (SAT) and renders:
  • A time slider (TimestampedGeoJson)
  • A static points layer with popups and colored styling per iconic taxon
  • The SAT line and a 2 km search buffer
  • A compact About box with project links

Notes
- Written to be robust against iNat rate limits (backoff + jitter) and 414 (URI too long) on polygons.
- Keeps Swedish labels/comments where helpful.
- Avoids duplicate code, fixes missing imports, and guards JSON serialization.

Prereqs: pip install folium shapely requests tqdm branca jinja2
Input:   SAT_full.geojson (trail geometry)
Output:  output/179_inat_taxa_layers_colored.html
"""
from __future__ import annotations

import html
import json
import random
import time
from collections.abc import Mapping
from dataclasses import dataclass
from datetime import date, datetime as dt, timedelta
from pathlib import Path
from typing import Any, Iterable

import requests
import folium
from folium.plugins import TimestampedGeoJson
from shapely.geometry import (
    LineString,
    MultiLineString,
    Point,
    Polygon,
    shape,
    mapping,
)
from shapely.ops import unary_union

# ---- Progressbar (fallback till no-op) ----
try:
    from tqdm.auto import tqdm  # type: ignore
except Exception:  # pragma: no cover
    def tqdm(x: Iterable, **_: Any) -> Iterable:
        return x

# ============================ Konstanter ============================
INAT_URL = "https://api.inaturalist.org/v1/observations"
TRAIL_FILE = Path("SAT_full.geojson")
OUTPUT_HTML = Path("output/179_inat_taxa_layers_colored.html")
BUFFER_DEG = 0.02  # ~2000 m vid 59–60°N

TAXON_COLORS: dict[str, str] = {
    "Plantae": "#2b8a3e",
    "Aves": "#1d4ed8",
    "Mammalia": "#78350f",
    "Reptilia": "#059669",
    "Amphibia": "#14b8a6",
    "Insecta": "#eab308",
    "Arachnida": "#7f1d1d",
    "Mollusca": "#9333ea",
    "Animalia": "#6b7280",
    "Fungi": "#6b21a8",
}

TAXA_GROUPS: list[tuple[str, str]] = [
    ("Aves", "Birds"),
    ("Plantae", "Plants"),
    ("Mammalia", "Mammals"),
    ("Fungi", "Fungi"),
    ("Insecta", "Insects"),
    ("Amphibia", "Amphibians"),
    ("Reptilia", "Reptiles"),
]

# ============================ Hjälpfunktioner ============================

def _get_with_backoff(url: str, params: dict[str, Any], *, timeout: int = 30, tries: int = 5, base_wait: float = 2.0) -> requests.Response:
    """GET med exponential backoff + jitter. Hanterar 429/5xx.
    Höjer för övriga statuskoder.
    """
    attempt = 0
    while True:
        r = requests.get(url, params=params, timeout=timeout)
        if r.status_code in (429, 500, 502, 503, 504):
            attempt += 1
            if attempt >= tries:
                r.raise_for_status()
            wait = (base_wait ** attempt) + random.uniform(0, 1)
            print("\tBackoff:", f"{wait:.2f}s", "for", r.status_code)
            time.sleep(wait)
            continue
        r.raise_for_status()
        return r


def _polygon_param_from_poly(poly: Polygon, *, simplify_tol: float = 0.0002, step: int = 1) -> str:
    p = poly.simplify(simplify_tol, preserve_topology=True)
    ring = list(p.exterior.coords)[::step]
    if ring[0] != ring[-1]:
        ring.append(ring[0])
    return ",".join(f"{lng},{lat}" for (lng, lat) in ring)


def _bbox_params_from_poly(poly: Polygon) -> dict[str, float]:
    minx, miny, maxx, maxy = poly.bounds
    return {"swlng": minx, "swlat": miny, "nelng": maxx, "nelat": maxy}


def _obs_in_geom(o: dict[str, Any], geom: Polygon) -> bool:
    gj = o.get("geojson")
    if not (isinstance(gj, dict) and isinstance(gj.get("coordinates"), (list, tuple)) and len(gj["coordinates"]) >= 2):
        return False
    lon, lat = gj["coordinates"][:2]
    try:
        pt = Point(lon, lat)
        return geom.contains(pt) or geom.touches(pt)
    except Exception:
        return False


def _obs_to_feature(o: dict[str, Any], *, color: str = "#3388ff", radius: int = 4, fill_opacity: float = 0.6) -> dict[str, Any] | None:
    gj = o.get("geojson") or {}
    if not (isinstance(gj.get("coordinates"), (list, tuple)) and len(gj["coordinates"]) >= 2):
        return None
    lon, lat = gj["coordinates"][:2]

    # Preferera full ISO-tid; fallback till datum
    obs_time = o.get("time_observed_at") or o.get("observed_on") or o.get("created_at")
    if not obs_time:
        return None
    if len(obs_time) > 10:
        obs_time = obs_time[:10]

    taxon = o.get("taxon") or {}
    cname = taxon.get("preferred_common_name") or taxon.get("name") or "okänd"
    sname = taxon.get("name") or ""
    oid = o.get("id", "")
    url = f"https://www.inaturalist.org/observations/{oid}"
    api = f"https://api.inaturalist.org/v1/observations/{oid}"

    img = ""
    photos = o.get("photos") or []
    if photos:
        raw = photos[0].get("url", "")
        if raw:
            img = raw.replace("square", "medium")

    popup = f"<b>{html.escape(cname)}</b><br><i>{html.escape(sname)}</i><br>{html.escape(obs_time)}<br>"
    if img:
        popup += f"<img src='{html.escape(img)}' width='220'><br>"
    popup += f"<a href='{html.escape(url)}' target='_blank'>Visa i iNaturalist</a> · "
    popup += f"<a href='{html.escape(api)}' target='_blank'>API JSON</a>"

    return {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lon, lat]},
        "properties": {
            "time": obs_time,  # för tidsreglaget
            "popup": popup,
            "icon": "circle",
            "iconstyle": {"color": color, "fillColor": color, "fillOpacity": 0.8},
            "style": {"radius": radius, "color": color, "fillColor": color, "fillOpacity": fill_opacity},
            "radius": radius,
        },
    }


def _to_feature_collection(obs: list[dict[str, Any]], *, clip_geom: Polygon, color: str, desc: str, radius: int = 4, fill_opacity: float = 0.6) -> tuple[dict[str, Any], list[tuple[float, float]], int]:
    feats: list[dict[str, Any]] = []
    bounds: list[tuple[float, float]] = []
    skipped = 0
    for o in tqdm(obs, desc=desc, leave=False):
        if not _obs_in_geom(o, clip_geom):
            skipped += 1
            continue
        f = _obs_to_feature(o, color=color, radius=radius, fill_opacity=fill_opacity)
        if f:
            feats.append(f)
            lon, lat = f["geometry"]["coordinates"]
            bounds.append((lat, lon))
        else:
            skipped += 1
    return {"type": "FeatureCollection", "features": feats}, bounds, skipped


# ============================ iNat fetchers ============================

def _inat_query_for_polygon(
    poly: Polygon,
    params_base: dict[str, Any],
    *,
    per_page: int = 200,
    max_pages: int = 5,
    pause: float = 0.2,
    pbar: Any | None = None,
    ext_bbox: tuple[float, float, float, float] | None = None,
) -> list[dict[str, Any]]:
    """Kör iNat mot polygon. Adaptiv förenkling + bbox-fallback.
    Om ext_bbox ges (swlat, swlng, nelat, nelng) används den i fallbacken.
    """
    results: list[dict[str, Any]] = []
    params = dict(params_base)
    params.update({
        "verifiable": "true",
        "order_by": "observed_on",
        "order": "desc",
        "per_page": per_page,
    })

    # 1) Adaptiv polygon – prova olika förenklingar och sampling-steps
    for simplify_tol in (0.0002, 0.0004, 0.0008, 0.0016):
        for step in (1, 2, 3, 4):
            polygon_param = _polygon_param_from_poly(poly, simplify_tol=simplify_tol, step=step)
            params_poly = dict(params, polygon=polygon_param)
            try:
                r = _get_with_backoff(INAT_URL, dict(params_poly, page=1), timeout=30)
                if r.status_code == 414:
                    raise requests.HTTPError("414 URI Too Large (polygon)", response=r)
                data = r.json()
                batch = data.get("results", [])
                results.extend(batch)
                if pbar:
                    pbar.update(1)
                for page in range(2, max_pages + 1):
                    r = _get_with_backoff(INAT_URL, dict(params_poly, page=page), timeout=30)
                    b2 = r.json().get("results", [])
                    results.extend(b2)
                    if pbar:
                        pbar.update(1)
                    if len(b2) < per_page:
                        break
                    time.sleep(pause)
                return results
            except requests.HTTPError as e:
                if getattr(e, "response", None) is None or e.response.status_code != 414:
                    raise
                # annars: prova nästa förenkling

    # 2) Fallback till bbox
    if ext_bbox is not None:
        swlat, swlng, nelat, nelng = ext_bbox
        params_bbox = dict(params, swlat=swlat, swlng=swlng, nelat=nelat, nelng=nelng)
    else:
        params_bbox = dict(params, **_bbox_params_from_poly(poly))

    for page in range(1, max_pages + 1):
        r = _get_with_backoff(INAT_URL, dict(params_bbox, page=page), timeout=30)
        b = r.json().get("results", [])
        results.extend(b)
        if pbar:
            pbar.update(1)
        if len(b) < per_page:
            break
        time.sleep(pause)
    return results


def fetch_inat_all_parts(
    geom_like: Any,
    *,
    iconic_taxa: str | None = None,
    d1: str | None = None,
    d2: str | None = None,
    per_page: int = 200,
    max_pages: int = 5,
    pause: float = 0.2,
    show_progress: bool = True,
    desc: str = "Fetch",
    swlat: float | None = None,
    swlng: float | None = None,
    nelat: float | None = None,
    nelng: float | None = None,
) -> list[dict[str, Any]]:
    """Tar LineString/Polygon/MultiPolygon, buffrar vid behov, loopar delpolygoner,
    deduplar observationer och visar progressbar.

    NYTT: valfri extern bbox (swlat, swlng, nelat, nelng) som insnävning.
    """
    g = geom_like if hasattr(geom_like, "geom_type") else shape(geom_like)
    if g.geom_type == "LineString":
        g = g.buffer(0.02)  # ~200 m om du vill, men vi kör ändå extern bbox
    elif g.geom_type not in ("Polygon", "MultiPolygon"):
        g = unary_union(g)

    polys: list[Polygon]
    if isinstance(g, Polygon):
        polys = [g]
    else:
        polys = [p for p in g.geoms if isinstance(p, Polygon) and p.area > 1e-10]

    base: dict[str, Any] = {}
    if iconic_taxa:
        base["iconic_taxa"] = iconic_taxa
    if d1:
        base["d1"] = d1
    if d2:
        base["d2"] = d2
    base["geoprivacy"] = "open"

    if all(v is not None for v in (swlat, swlng, nelat, nelng)):
        base.update({"swlat": swlat, "swlng": swlng, "nelat": nelat, "nelng": nelng})
        ext_bbox = (swlat, swlng, nelat, nelng)  # type: ignore[assignment]
    else:
        ext_bbox = None

    total_steps = len(polys) * max_pages
    pbar = tqdm(total=total_steps, desc=desc, unit="page", leave=False) if show_progress else None

    all_obs: dict[int, dict[str, Any]] = {}
    try:
        for poly in polys:
            part = _inat_query_for_polygon(
                poly,
                base,
                per_page=per_page,
                max_pages=max_pages,
                pause=pause,
                pbar=pbar,
                ext_bbox=ext_bbox,
            )
            for o in part:
                oid = o.get("id")
                if oid is not None:
                    all_obs[oid] = o
    finally:
        if pbar:
            pbar.close()
    return list(all_obs.values())


# ============================ About-box ============================

def add_about_box(
    m: folium.Map,
    *,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
    offset_px: tuple[int, int] = (10, 54),  # (top, left) – 54 px bredvid zoom
) -> None:
    """Superenkel, robust About-box som kan minimeras och minns sitt läge (localStorage)."""
    if created_date is None:
        created_date = dt.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id = f"sat-about-{map_dom_id}"
    header_id = f"{box_id}-hdr"
    issue_url = f"https://github.com/{repo}/issues/{issue_number}"
    top, left = offset_px
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]

    links_html = "".join(
        f'<div><a href="{html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {html.escape(t)}</a></div>'
        for t, u in links
    )

    from string import Template as _Tpl
    tpl = _Tpl(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; top: ${top}px; left: ${left}px;
    background: rgba(255,255,255,0.97); border: 2px solid #666; border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.25); font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    min-width: 240px; max-width: 320px; pointer-events: auto; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700; display: flex; align-items: center; gap: 6px; user-select: none; background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>
<div id="${box_id}" class="sat-about ${collapsed_class}">
  <div id="${header_id}" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="${issue_url}" target="_blank">#${issue_number}</a>&nbsp;&nbsp; Map: ${map_name}</div>
    <div>Created: ${created_date}</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">${links_html}</div>
  </div>
</div>
<script>
(function(){
  var boxId = "${box_id}";
  var hdrId = "${header_id}";
  var storageKey = "satAboutCollapsed_${map_dom_id}_#${issue_number}";
  function setCollapsed(box, collapsed) {
    if (!box) return;
    if (collapsed) box.classList.add("sat-about-collapsed");
    else box.classList.remove("sat-about-collapsed");
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }
  function init(){
    var box = document.getElementById(boxId);
    var hdr = document.getElementById(hdrId);
    if (!box || !hdr) return;
    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}
    hdr.addEventListener("click", function(e){ e.stopPropagation(); setCollapsed(box, !box.classList.contains("sat-about-collapsed")); });
  }
  if (document.readyState === "loading") document.addEventListener("DOMContentLoaded", init); else init();
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=html.escape(map_name),
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
        top=top,
        left=left,
    )
    m.get_root().html.add_child(folium.Element(html_code))


# ============================ Huvudkörning ============================

def main() -> None:
    if not TRAIL_FILE.exists():
        raise FileNotFoundError("Hittar inte SAT_full.geojson.")

    with TRAIL_FILE.open(encoding="utf-8") as f:
        gj = json.load(f)

    # --- Extrahera linjesegment ---
    lines: list[LineString] = []
    for feat in gj.get("features", []):
        geom = shape(feat.get("geometry"))
        if isinstance(geom, LineString) and len(geom.coords) >= 2:
            lines.append(geom)
        elif isinstance(geom, MultiLineString):
            for seg in geom.geoms:
                if len(seg.coords) >= 2:
                    lines.append(seg)

    if not lines:
        raise ValueError("SAT_full.geojson innehåller inga linjesegment.")

    trail = unary_union(lines)
    center_lat, center_lon = trail.centroid.y, trail.centroid.x

    # --- Buffert + bbox ---
    search_geom = trail.buffer(BUFFER_DEG)
    if hasattr(search_geom, "simplify"):
        search_geom = search_geom.simplify(0.0002, preserve_topology=True)

    minx, miny, maxx, maxy = search_geom.bounds
    swlng, swlat, nelng, nelat = minx, miny, maxx, maxy

    # --- Datumintervall (senaste 10 dagar) ---
    d2 = date.today().isoformat()
    d1 = (date.today() - timedelta(days=10)).isoformat()
    print("Datumintervall:", d1, "→", d2)

    # --- Hämta observationer per taxa ---
    all_groups: dict[str, list[dict[str, Any]]] = {}
    for iconic_taxa, desc in TAXA_GROUPS:
        all_groups[iconic_taxa] = fetch_inat_all_parts(
            search_geom,
            iconic_taxa=iconic_taxa,
            d1=d1,
            d2=d2,
            per_page=200,
            max_pages=5,
            show_progress=True,
            desc=desc,
            swlat=swlat,
            swlng=swlng,
            nelat=nelat,
            nelng=nelng,
        )

    # --- Bygg karta ---
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10, control_scale=True)

    folium.GeoJson(mapping(trail), name="SAT-led", style_function=lambda _x: {"color": "#1e40af", "weight": 3}).add_to(m)
    folium.GeoJson(
        mapping(search_geom),
        name="Sökzon (2000 m)",
        style_function=lambda _x: {"color": "#3388ff", "weight": 1, "fillOpacity": 0.05},
    ).add_to(m)

    # --- Samla alla features till tidslager + statiskt lager ---
    all_bounds: list[tuple[float, float]] = []
    all_features: list[dict[str, Any]] = []
    total_skipped = 0

    for tx, obs in tqdm(all_groups.items(), desc="Bygger features", total=len(all_groups)):
        color = TAXON_COLORS.get(tx, "#3388ff")
        fc, bounds, skipped = _to_feature_collection(obs, clip_geom=search_geom, color=color, desc=f"{tx} → features", radius=4)
        all_features.extend(fc["features"])
        all_bounds.extend(bounds)
        total_skipped += skipped

    fc_all = {"type": "FeatureCollection", "features": all_features}
    n_features = len(fc_all["features"])
    print(f"Features in time layer: {n_features}")
    if not n_features:
        print("No features → widen date range, loosen buffer/bbox, or fetch fewer taxa.")

    # 1) Tidslager
    days_total = 3650  # ~10 år
    total_ms = 10000   # ~10 s
    transition_time = max(1, int(total_ms / days_total))

    TimestampedGeoJson(
        fc_all,
        period="P1D",
        duration="P1Y",
        auto_play=True,
        loop=True,
        transition_time=transition_time,
    ).add_to(m)

    # 2) Statiskt lager (alla punkter)
    # 2) Statiskt lager (alla punkter) — FIXED
    fg_all = folium.FeatureGroup(name="Alla observationer", show=True)
    
    for f in fc_all["features"]:
        lon, lat = f["geometry"]["coordinates"]
        props = f.get("properties", {})
        style = props.get("style", {})
        radius = props.get("radius", 3)
        color = style.get("color", "black")
        fill_color = style.get("fillColor", color)
        fill_opacity = style.get("fillOpacity", 0.4)
    
        folium.CircleMarker(
            location=(lat, lon),
            radius=radius,
            color=color,
            fill=True,
            fill_color=fill_color,
            fill_opacity=fill_opacity,
            weight=1,
            popup=folium.Popup(props.get("popup"), max_width=350),
        ).add_to(fg_all)
    
    fg_all.add_to(m)


    # 3) Per-taxon lager + count
    taxon_counts: dict[str, int] = {}
    for tx, obs in all_groups.items():
        color = TAXON_COLORS.get(tx, "#888")
        fc, _bounds, skipped = _to_feature_collection(obs, clip_geom=search_geom, color=color, desc=f"{tx} → features", radius=4)
        n = len(fc["features"])
        taxon_counts[tx] = n
        fg = folium.FeatureGroup(name=f"{tx} ({n})", show=True)
        folium.GeoJson(
            fc,
            name=tx,
            style_function=lambda _x, c=color: {"color": c, "weight": 2, "fillOpacity": 0.3},
            tooltip=folium.GeoJsonTooltip(fields=["time"], aliases=["Datum"], labels=True, sticky=False),
        ).add_to(fg)
        fg.add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)

    # --- Legend ---
    legend_html = """
<div style="position: fixed; bottom: 50px; right: 50px; width: 170px; background: white; border:2px solid grey; z-index:9999; font-size:14px; padding: 10px;">
  <b>Taxa</b><br>
  <i style="color:#2b8a3e;">●</i> Växter<br>
  <i style="color:#1d4ed8;">●</i> Fåglar<br>
  <i style="color:#78350f;">●</i> Däggdjur<br>
  <i style="color:#6b21a8;">●</i> Svampar<br>
  <i style="color:#eab308;">●</i> Insekter<br>
  <i style="color:#14b8a6;">●</i> Groddjur<br>
  <i style="color:#059669;">●</i> Kräldjur<br>
  <i style="color:#7f1d1d;">●</i> Spindeldjur<br>
  <i style="color:#9333ea;">●</i> Blötdjur
</div>
"""
    m.get_root().html.add_child(folium.Element(legend_html))

    # --- Auto-zoom ---
    if all_bounds:
        m.fit_bounds(all_bounds)

    # --- About box ---
    add_about_box(m, issue_number=179, map_name="SAT iNaturalist")

    # --- Spara ---
    OUTPUT_HTML.parent.mkdir(parents=True, exist_ok=True)
    m.save(str(OUTPUT_HTML))
    print(f"✅ Sparade: {OUTPUT_HTML}")


if __name__ == "__main__":
    main()


Datumintervall: 2025-09-13 → 2025-09-23


Birds:   0%|          | 0/75 [00:00<?, ?page/s]

Plants:   0%|          | 0/75 [00:00<?, ?page/s]

Mammals:   0%|          | 0/75 [00:00<?, ?page/s]

Fungi:   0%|          | 0/75 [00:00<?, ?page/s]

...

Fungi → features:   0%|          | 0/591 [00:00<?, ?it/s]

Insecta → features:   0%|          | 0/93 [00:00<?, ?it/s]

Amphibia → features:   0%|          | 0/5 [00:00<?, ?it/s]

Reptilia → features:   0%|          | 0/8 [00:00<?, ?it/s]

AssertionError: The field time is not available in the data. Choose from: ().

In [ ]:
end_time = time.time()
duration = end_time - start_time
print(f"GeoJSON saved to {OUT}")
print(f"Finished in {duration:.2f} seconds.")
